In [1]:
import pandas as pd
import numpy as np
import os,time,fnmatch

In [2]:
import sys
sys.path.insert(0,'/home/bhabas/catkin_ws/src/crazyflie_simulation/src/crazyflie_gazebo_sim/src/utility')
from data_analysis import DataFile
dataPath = '/home/bhabas/catkin_ws/src/crazyflie_simulation/local_files/data/'

In [3]:
## ITERATE OVER ALL COMBINATIONS
temp_list = []
for vz_d in np.arange(1.5,4.0,0.5): # [1.5,3.5]
    for vx_d in np.arange(0,3.5,0.5):   # [0,3.0]
        # print(f"{vz_d}|{vx_d}")
        temp_list.append([vz_d,vx_d])
test_arr = np.asarray(temp_list)

In [4]:
d = []
for vz_d,vx_d in test_arr:
    for file in os.listdir(dataPath):
        csv = os.path.join(dataPath,file)
        if fnmatch.fnmatch(file,f"Vz_{vz_d}--Vx_{vx_d}--trial_*.csv"):

            trial = DataFile(csv)
            trial_num = file[-5]

            landing_rate = trial.landing_rate()
            policy,_ = trial.grab_finalPolicy()
            RREV_trig,G1,G2 = policy

            d.append( (vz_d,vx_d,trial_num,landing_rate,RREV_trig,G1,G2))

policy_df = pd.DataFrame(d, columns=('vz_d','vx_d','trial_num','landing_rate','RREV_trigger','G1','G2'))

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib widget


In [6]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vz_d'],policy_df['vx_d'],policy_df['RREV_trigger'])

ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('RREV')

ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vz_d'],policy_df['vx_d'],policy_df['G2'])

ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('G2')

ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
## Landing Rate Plot
from matplotlib import cm

grouped = policy_df.groupby(['vz_d','vx_d'])['landing_rate'].mean().reset_index()

fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.plot_trisurf(grouped['vz_d'],grouped['vx_d'],grouped['landing_rate'],cmap=cm.jet)
ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('LR')
ax.set_title('Average Landing Rate for Final Two Episodes')

ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …